In [195]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import GHMCMove, MCMCSampler, MCRotationMove, BaseIntegratorMove, IntegratorMoveError
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from openmmtools.integrators import AlchemicalNonequilibriumLangevinIntegrator, LangevinIntegrator
import numpy as np
import copy, sys

In [20]:
from openmmtools import testsystems, alchemy
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False,
            disable_alchemical_dispersion_correction=True,
            alchemical_pme_treatment='direct-space')
test = testsystems.AlanineDipeptideVacuum()
alanine_vacuum = testsystems.AlanineDipeptideVacuum().system
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(22))
alanine_alchemical_system = factory.create_alchemical_system(reference_system=alanine_vacuum,
                                                           alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(alanine_alchemical_system)

In [22]:
sampler_state = SamplerState(test.positions)

In [23]:
ghmc_move = GHMCMove(timestep=1.0*unit.femtosecond, n_steps=50)

In [24]:
sampler = MCMCSampler(compound_state, sampler_state, move=ghmc_move)

In [25]:
sampler.minimize()
sampler.run(n_iterations=2)

# Trial with compound objects

In [206]:
tol = testsystems.TolueneImplicitGBn2()
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False,
            disable_alchemical_dispersion_correction=True,
            alchemical_pme_treatment='direct-space')
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(15),
                           softcore_alpha=0.5,
                           softcore_a=1,
                           softcore_b=1,
                           softcore_c=6,
                           softcore_beta=0.0,
                           softcore_d=1,
                           softcore_e=1,
                           softcore_f=2,
                           annihilate_electrostatics=True,
                           annihilate_sterics=False)
alchemical_atoms = list(alchemical_region.alchemical_atoms)
tol_alchemical_system = factory.create_alchemical_system(reference_system=tol.system, alchemical_regions=alchemical_region)
tol_alchemical_state = alchemy.AlchemicalState.from_system(tol_alchemical_system)

//anaconda/envs/molssi/lib/python3.5/site-packages/simtk/openmm/app/internal/amber_file_parser.py:1017: UserWarning: Non-optimal GB parameters detected for GB model GBn2
  'Non-optimal GB parameters detected for GB model %s' % gbmodel)


In [212]:
from openmmtools import testsystems, alchemy
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False,
            disable_alchemical_dispersion_correction=True,
            alchemical_pme_treatment='direct-space')
ala = testsystems.AlanineDipeptideVacuum()
alanine_vacuum = testsystems.AlanineDipeptideVacuum().system
alchemical_atoms = list(alchemical_region.alchemical_atoms)
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(22))
alanine_alchemical_system = factory.create_alchemical_system(reference_system=alanine_vacuum,
                                                           alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(alanine_alchemical_system)

In [213]:
thermo_state = ThermodynamicState(system=alanine_alchemical_system, temperature=300*unit.kelvin)
compound_state = CompoundThermodynamicState(thermo_state, composable_states=[alchemical_state])
sampler_state = SamplerState(positions=ala.positions)
print(compound_state.lambda_sterics)
print(compound_state.lambda_electrostatics)

1.0
1.0


In [214]:
import logging
from openmmtools.utils import SubhookedABCMeta, Timer
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

class NCMCSampler(object):

    def __init__(self, thermodynamic_state, sampler_state, move):
        # Make a deep copy of the state so that initial state is unchanged.
        self.thermodynamic_state = copy.deepcopy(thermodynamic_state)
        self.sampler_state = copy.deepcopy(sampler_state)
        self.move = move

    def run(self, n_iterations=1):
        """
        Run the sampler for a specified number of iterations.

        Parameters
        ----------
        n_iterations : int
            Number of iterations of the sampler to run.

        """
        # Apply move for n_iterations.
        for iteration in range(n_iterations):
            self.move.apply(self.thermodynamic_state, self.sampler_state)

    def minimize(self,
                 tolerance=1.0 * unit.kilocalories_per_mole / unit.angstroms,
                 max_iterations=100,
                 context_cache=None):
        """Minimize the current configuration.

        Parameters
        ----------
        tolerance : simtk.unit.Quantity, optional
            Tolerance to use for minimization termination criterion (units of
            energy/(mole*distance), default is 1*kilocalories_per_mole/angstroms).
        max_iterations : int, optional
            Maximum number of iterations to use for minimization. If 0, the minimization
            will continue until convergence (default is 100).
        context_cache : openmmtools.cache.ContextCache, optional
            The ContextCache to use for Context creation. If None, the global cache
            openmmtools.cache.global_context_cache is used (default is None).

        """
        if context_cache is None:
            context_cache = cache.global_context_cache

        timer = Timer()

        # Use LocalEnergyMinimizer
        timer.start("Context request")
        integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context, integrator = context_cache.get_context(self.thermodynamic_state, integrator)
        self.sampler_state.apply_to_context(context)
        logger.debug("LocalEnergyMinimizer: platform is %s" % context.getPlatform().getName())
        logger.debug("Minimizing with tolerance %s and %d max. iterations." % (tolerance, max_iterations))
        timer.stop("Context request")

        timer.start("LocalEnergyMinimizer minimize")
        openmm.LocalEnergyMinimizer.minimize(context, tolerance, max_iterations)
        timer.stop("LocalEnergyMinimizer minimize")
        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                 enforcePeriodicBox=self.thermodynamic_state.is_periodic)


        potential_energy = context_state.getPotentialEnergy()
        print(potential_energy)
        # Retrieve data.
        self.sampler_state.update_from_context(context)

        #timer.report_timing()


In [303]:
class NCMCMove(BaseIntegratorMove):

    def __init__(self, timestep=1.0*unit.femtosecond, collision_rate=1.0/unit.picoseconds,
                 n_steps=1000, temperature=300*unit.kelvin, **kwargs):
        super(NCMCMove, self).__init__(n_steps=n_steps, **kwargs)
        self.timestep = timestep
        self.temperature = temperature
        self.collision_rate = collision_rate
        self.n_accepted = 0  # Number of accepted steps.
        self.n_proposed = 0  # Number of attempted steps.

    @property
    def fraction_accepted(self):
        """Ratio between accepted over attempted moves (read-only).

        If the number of attempted steps is 0, this is numpy.NaN.

        """
        if self.n_proposed == 0:
            return np.NaN
        # TODO drop the casting when stop Python2 support
        return float(self.n_accepted) / self.n_proposed

    @property
    def statistics(self):
        """The acceptance statistics as a dictionary."""
        return dict(n_accepted=self.n_accepted, n_proposed=self.n_proposed)

    @statistics.setter
    def statistics(self, value):
        self.n_accepted = value['n_accepted']
        self.n_proposed = value['n_proposed']

    def reset_statistics(self):
        """Reset the internal statistics of number of accepted and attempted moves."""
        self.n_accepted = 0
        self.n_proposed = 0

    def apply(self, thermodynamic_state, sampler_state):
            """Propagate the state through the integrator.

            This updates the SamplerState after the integration. It also logs
            benchmarking information through the utils.Timer class.

            Parameters
            ----------
            thermodynamic_state : openmmtools.states.ThermodynamicState
               The thermodynamic state to use to propagate dynamics.
            sampler_state : openmmtools.states.SamplerState
               The sampler state to apply the move to. This is modified.

            See Also
            --------
            openmmtools.utils.Timer

            """
            move_name = self.__class__.__name__  # shortcut
            timer = Timer()

            # Check if we have to use the global cache.
            if self.context_cache is None:
                context_cache = cache.global_context_cache
            else:
                context_cache = self.context_cache

            # Create integrator.
            integrator = self._get_integrator(thermodynamic_state)
            #integrator.setCurrentIntegrator(0)
            
            # Create context.
            timer.start("{}: Context request".format(move_name))
            context, integrator = context_cache.get_context(thermodynamic_state, integrator)
            print('SamplerContext compat:', sampler_state.is_context_compatible(context))
            integrator.setCurrentIntegrator(0)
            timer.stop("{}: Context request".format(move_name))
            logger.debug("{}: Context obtained, platform is {}".format(
                move_name, context.getPlatform().getName()))

            # Perform the integration.
            for attempt_counter in range(self.n_restart_attempts + 1):

                #If we reassign velocities, we can ignore the ones in sampler_state.
                sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
                if self.reassign_velocities:
                   context.setVelocitiesToTemperature(thermodynamic_state.temperature)

                # Subclasses may implement _before_integration().
                self._before_integration(context, thermodynamic_state)

                try:
                    # Run dynamics.
                    timer.start("{}: step({})".format(move_name, self.n_steps))
                    #integrator.pretty_print()
                    integrator.step(self.n_steps)
                    
                except Exception:
                    # Catches particle positions becoming nan during integration.
                    restart = True
                else:
                    timer.stop("{}: step({})".format(move_name, self.n_steps))

                    # We get also velocities here even if we don't need them because we
                    # will recycle this State to update the sampler state object. This
                    # way we won't need a second call to Context.getState().
                    context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                     enforcePeriodicBox=thermodynamic_state.is_periodic)

                    # Check for NaNs in energies.
                    potential_energy = context_state.getPotentialEnergy()
                    print('potential_energy', potential_energy)
                    restart = np.isnan(potential_energy.value_in_unit(potential_energy.unit))

                # Restart the move if we found NaNs.
                if restart:
                    err_msg = ('Potential energy is NaN after {} attempts of integration '
                               'with move {}'.format(attempt_counter, self.__class__.__name__))

                    # If we are on our last chance before crash, try to re-initialize context
                    if attempt_counter == self.n_restart_attempts - 1:
                        logger.error(err_msg + ' Trying to reinitialize Context as a last-resort restart attempt...')
                        context.reinitialize()
                        sampler_state.apply_to_context(context)
                        thermodynamic_state.apply_to_context(context)
                    # If we have hit the number of restart attempts, raise an exception.
                    elif attempt_counter == self.n_restart_attempts:
                        # Restore the context to the state right before the integration.
                        sampler_state.apply_to_context(context)
                        logger.error(err_msg)
                        raise IntegratorMoveError(err_msg, self, context)
                    else:
                        logger.warning(err_msg + ' Attempting a restart...')
                else:
                    break

            # Subclasses can read here info from the context to update internal statistics.
            self._after_integration(context, thermodynamic_state)

            # Updated sampler state.
            timer.start("{}: update sampler state".format(move_name))
            # This is an optimization around the fact that Collective Variables are not a part of the State,
            # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
            # the State.
            # Update everything but the collective variables from the State object
            sampler_state.update_from_context(context_state, ignore_collective_variables=True)
            # Update only the collective variables from the Context
            sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
                                              ignore_collective_variables=False)
            timer.stop("{}: update sampler state".format(move_name))

            #timer.report_timing()

    def __getstate__(self):
        serialization = super(NCMCMove, self).__getstate__()
        serialization['timestep'] = self.timestep
        serialization['temperature'] = self.temperature
        serialization['collision_rate'] = self.collision_rate
        serialization.update(self.statistics)
        return serialization

    def __setstate__(self, serialization):
        super(NCMCMove, self).__setstate__(serialization)
        self.timestep = serialization['timestep']
        self.temperature = serialization['temperature']
        self.collision_rate = serialization['collision_rate']
        self.statistics = serialization

    def _get_integrator(self, thermodynamic_state):
        """Implement BaseIntegratorMove._get_integrator()."""
        # Store lastly generated integrator to collect statistics.
        alchemical_functions={ 'lambda_sterics' : 'min(1, (1/0.3)*abs(lambda-0.5))',
                               'lambda_electrostatics' : 'step(0.2-lambda) - 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)' }
        ncmc_integrator = AlchemicalNonequilibriumLangevinIntegrator(alchemical_functions, 
                                                                     splitting='O { V R H R V } O',
                                                                     temperature=self.temperature,
                                                                     collision_rate=self.collision_rate,
                                                                     timestep=self.timestep,
                                                                     nsteps_neq=self.n_steps)
        #integrator = LangevinIntegrator(temperature=self.temperature, 
        #                                timestep=self.timestep,
        #                               collision_rate=self.collision_rate)
        #integrator = openmm.VerletIntegrator(self.timestep)
        compound_integrator = openmm.CompoundIntegrator()
        compound_integrator.addIntegrator(ncmc_integrator)
        #compound_integrator.addIntegrator(integrator)
        return compound_integrator

#     def _before_integration(self, context, thermodynamic_state):
#         """Execute code after Context creation and before integration."""
#         integrator = context.getIntegrator()
#         print('BEFORE INTEGRATION')
#         integrator.setCurrentIntegrator(1)
#         print(integrator.getCurrentIntegrator())
    
    def _after_integration(self, context, thermodynamic_state):
        """Implement BaseIntegratorMove._after_integration()."""
        # Check if we have to use the global cache.
        if self.context_cache is None:
            context_cache = cache.global_context_cache
        else:
            context_cache = self.context_cache
        
        context, integrator = context_cache.get_context(thermodynamic_state)
        print(integrator)
        #integrator = context.getIntegrator()
        integrator.setCurrentIntegrator(0)
        #integrator = integrator.getIntegrator(0)
        # Accumulate acceptance statistics.
        ncmc_global_variables = {integrator.getGlobalVariableName(index): index
                                 for index in range(integrator.getNumGlobalVariables())}
        print(ncmc_global_variables)
        n_accepted = integrator.getGlobalVariable(ncmc_global_variables['naccept'])
        n_proposed = integrator.getGlobalVariable(ncmc_global_variables['ntrials'])
        self.n_accepted += n_accepted
        self.n_proposed += n_proposed


In [304]:
cache.global_context_cache.time_to_live = 10

In [308]:
cache.global_context_cache.time_to_live = 10
ncmc_move = NCMCMove(timestep=1.0*unit.femtosecond, n_steps=500, context_cache=cache.global_context_cache)
sampler = NCMCSampler(compound_state, sampler_state, move=ncmc_move)
sampler.minimize(max_iterations=0)
sampler.run(n_iterations=1)

2018-09-11 23:16:31,142 | DEBUG : LocalEnergyMinimizer: platform is OpenCL
2018-09-11 23:16:31,143 | DEBUG : Minimizing with tolerance 1.0 kcal/(A mol) and 0 max. iterations.
-115.31268310546875 kJ/mol
SamplerContext compat: True
2018-09-11 23:16:31,336 | DEBUG : NCMCMove: Context obtained, platform is OpenCL
potential_energy -115.31268310546875 kJ/mol
<simtk.openmm.openmm.CompoundIntegrator; proxy of <Swig Object of type 'OpenMM::CompoundIntegrator *' at 0x1823debbd0> >


AttributeError: type object 'object' has no attribute '__getattr__'